In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
# Define some parameters
img_size = (224, 224)
batch_size = 32
epochs = 10
num_classes = 15

In [7]:
# Define the data generators
train_datagen = ImageDataGenerator(
                                   rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('split_data/train',
                                                    target_size=img_size,
                                                    batch_size=batch_size,
                                                    shuffle=False,
                                                    class_mode='categorical')

val_generator = val_datagen.flow_from_directory('split_data/val',
                                                target_size=img_size,
                                                batch_size=batch_size,
                                                shuffle=False,
                                                class_mode='categorical')


Found 5536 images belonging to 15 classes.
Found 686 images belonging to 15 classes.


In [8]:
# Load the pre-trained EfficientNet model with pre-trained weights from ImageNet
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(img_size[0], img_size[1], 3))

In [9]:
# Freeze the pre-trained layers to prevent their weights from being updated during training
for layer in base_model.layers:
    layer.trainable = False

In [10]:
# Add a new classification layer to the top of the pre-trained model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [11]:
# Compile the model with an appropriate optimizer and loss function
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
# Train the model on your training data
history = model.fit(train_generator, epochs=epochs, validation_data=val_generator)

Epoch 1/10
173/173 [==============================] - 364s 2s/step - loss: 2.6555 - accuracy: 0.1384 - val_loss: 2.3877 - val_accuracy: 0.1910
Epoch 2/10
173/173 [==============================] - 279s 2s/step - loss: 2.4152 - accuracy: 0.1738 - val_loss: 2.3841 - val_accuracy: 0.1910
Epoch 3/10
173/173 [==============================] - 234s 1s/step - loss: 2.4080 - accuracy: 0.1552 - val_loss: 2.3339 - val_accuracy: 0.1910
Epoch 4/10
173/173 [==============================] - 265s 2s/step - loss: 2.4021 - accuracy: 0.1685 - val_loss: 2.3734 - val_accuracy: 0.1910
Epoch 5/10
173/173 [==============================] - 273s 2s/step - loss: 2.3895 - accuracy: 0.1906 - val_loss: 2.3409 - val_accuracy: 0.1910
Epoch 6/10
173/173 [==============================] - 233s 1s/step - loss: 2.4067 - accuracy: 0.1877 - val_loss: 2.3402 - val_accuracy: 0.1633
Epoch 7/10
173/173 [==============================] - 234s 1s/step - loss: 2.3886 - accuracy: 0.1615 - val_loss: 2.3315 - val_accuracy: 0.1910

In [13]:
# Evaluate the model on your test data
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory('split_data/test',
                                                  target_size=img_size,
                                                  batch_size=batch_size,
                                                  class_mode='categorical')

test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

Found 706 images belonging to 15 classes.
23/23 [==============================] - 26s 1s/step - loss: 2.3668 - accuracy: 0.1884
Test accuracy: 0.1883852630853653


In [15]:
from keras.utils import to_categorical

y_pred = model.predict(test_generator)

# Compute LWLRAP score
y_true = to_categorical(test_generator.labels, num_classes=15)
lwlrAP = label_ranking_average_precision_score(y_true, y_pred)
print('LWLRAP score:', lwlrAP)

# Compute top-5 categorical accuracy
top5_accuracy = top_k_accuracy_score(test_generator.labels, y_pred, k=5)
print('Top-5 categorical accuracy:', top5_accuracy)

23/23 [==============================] - 25s 1s/step
LWLRAP score: 0.3966475332976744
Top-5 categorical accuracy: 0.6657223796033994
